## Langchain Expression Language Basics

-  LangChain Expression Language is that any two runnables can be "chained" together into sequences. 
- The output of the previous runnable's .invoke() call is passed as input to the next runnable.
- This can be done using the pipe operator (|), or the more explicit .pipe() method, which does the same thing.

- Type of LCEL Chains
    - SequentialChain
    - Parallel Chain
    - Router Chain
    - Chain Runnables
    - Custom Chain (Runnable Sequence)

In [9]:
from dotenv import load_dotenv

load_dotenv('./../.env')

True

### Sequential LCEL Chain

In [10]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import (
                                        SystemMessagePromptTemplate,
                                        HumanMessagePromptTemplate,
                                        PromptTemplate,
                                        ChatPromptTemplate
                                        )

base_url = "http://localhost:11434"
model = 'llama3.2:1b'

llm = ChatOllama(base_url=base_url, model=model)

llm

ChatOllama(model='llama3.2:1b', base_url='http://localhost:11434')

In [5]:
system = SystemMessagePromptTemplate.from_template('You are {school} teacher. You answer in short sentences.')

question = HumanMessagePromptTemplate.from_template('tell me about the {topics} in {points} points')

messages = [system, question]
template = ChatPromptTemplate(messages)

# question = template.invoke({'school': 'primary', 'topics': 'solar system', 'points': 5})

# response = llm.invoke(question)
# print(response.content)

In [6]:
chain = template | llm

In [7]:
response = chain.invoke({'school': 'primary', 'topics': 'solar system', 'points': 5})
print(response.content)

Here's a brief overview of the solar system:

1. The Sun is at the center, and it makes up 99% of the total mass.
2. The four planets closest to the Sun are Mercury, Mars, Venus, and Earth - they move around it in different paths.
3. Neptune and Uranus are gas giants outside our solar system's orbit, moving away from the Sun slowly.
4. Pluto is now classified as a dwarf planet, although it's still being studied by astronomers.
5. The Kuiper Belt contains many small icy objects like Eris and Haumea - they're thought to be remnants of the early days of our solar system.


In [8]:
response

AIMessage(content="Here's a brief overview of the solar system:\n\n1. The Sun is at the center, and it makes up 99% of the total mass.\n2. The four planets closest to the Sun are Mercury, Mars, Venus, and Earth - they move around it in different paths.\n3. Neptune and Uranus are gas giants outside our solar system's orbit, moving away from the Sun slowly.\n4. Pluto is now classified as a dwarf planet, although it's still being studied by astronomers.\n5. The Kuiper Belt contains many small icy objects like Eris and Haumea - they're thought to be remnants of the early days of our solar system.", additional_kwargs={}, response_metadata={'model': 'llama3.2:1b', 'created_at': '2025-01-03T03:54:39.5592872Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 9844633100, 'load_duration': 2730894400, 'prompt_eval_count': 46, 'prompt_eval_duration': 517000000, 'eval_count': 136, 'eval_duration': 5238000000}, id='run-ddb89d81-19d0-44b0-af98-f

In [14]:
from langchain_core.output_parsers import StrOutputParser

In [10]:
chain = template | llm | StrOutputParser()
response = chain.invoke({'school': 'primary', 'topics': 'solar system', 'points': 5})
print(response)

Here's an overview of the solar system in five key points:

1. **The Sun is at the Center**: The Sun is the star that makes life on Earth possible, and it's the center of our solar system.

2. **Four Planets in Our Solar System**: The four planets in our solar system are Mercury, Venus, Mars, and Jupiter. 

3. **Saturn and Uranus are Ice Giant Planets**: These two planets are made mostly of ice and rock, and they have some amazing ring systems that orbit around them.

4. **Neptune is the Farthest Planet from the Sun**: Neptune is the last planet in our solar system to visit after being left behind by a comet or asteroid that passed close to it many years ago.

5. **Mars has Water and Volcanoes**: Scientists think that Mars had water on its surface millions of years ago, which could be an important clue for future human missions to explore the Red Planet.


## Chaining Runnables (Chain Multiple Runnables)

In [11]:
chain

ChatPromptTemplate(input_variables=['points', 'school', 'topics'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['school'], input_types={}, partial_variables={}, template='You are {school} teacher. You answer in short sentences.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['points', 'topics'], input_types={}, partial_variables={}, template='tell me about the {topics} in {points} points'), additional_kwargs={})])
| ChatOllama(model='llama3.2:1b', base_url='http://localhost:11434')
| StrOutputParser()

In [12]:
analysis_prompt = ChatPromptTemplate.from_template('''analyze the following text: {response}.
                                                   You need to tell how difficult the text is to understand.
                                                   Answer in one sentence only.
                                                   ''')
fact_check_chain = analysis_prompt | llm | StrOutputParser()
output = fact_check_chain.invoke({'response': 'The sun is a star.'})
print(output)

The text appears to be simple and straightforward, with no complex vocabulary or abstract concepts, making it easy to understand.


In [40]:
composed_chain = {"response": chain} | analysis_prompt | llm | StrOutputParser()
output = composed_chain.invoke({'school': 'phd', 'topics': 'solar system', 'points': 5})
print(output)

The text is moderately complex and requires some background knowledge of astronomy, but it is still accessible to a general audience with basic understanding of the solar system.


## Parallel LCEL Chain

In [13]:
system = SystemMessagePromptTemplate.from_template('You are {school} teacher. You answer in short sentences.')

question = HumanMessagePromptTemplate.from_template('tell me about the {topics} in {points} points')

messages = [system, question]

fact_chain = template | llm | StrOutputParser()

output = fact_chain.invoke({'school': 'phd', 'topics': 'solar system', 'points': 2})

print(output)

The sun is a massive star at the center of our solar system, with eight planets orbiting around it:
 
1. Mercury and Venus are close to the sun and get extremely hot during their orbits.
2. Earth and Mars have conditions that can support life; however, there are risks associated with living in these environments.


In [14]:
question = HumanMessagePromptTemplate.from_template('write a poem on {topics} in {sentences} lines.')

messages = [system, question]
template = ChatPromptTemplate(messages)
poem_chain = template | llm | StrOutputParser()

output = poem_chain.invoke({'school': 'phd', 'topics': 'solar system', 'sentences': 2})

print(output)

The sun at center, planets spin,
Mercury to Neptune, each one within.


In [15]:
from langchain_core.runnables import RunnableParallel

In [16]:
chain = RunnableParallel(fact = fact_chain, poem = poem_chain)

In [17]:
output = chain.invoke({'school': 'phd', 'topics': 'solar system', 'points': 2, 'sentences': 2})

{'fact': "Here's an overview of the solar system:\n\n• Our solar system consists of eight planets, including Mercury, Mars, Earth, and Jupiter, orbiting around the Sun.\n\n• The four outer planets (Jupiter, Saturn, Uranus, and Neptune) are gas giants with diverse compositions and unique features.", 'poem': 'Planets orbit Earth, a celestial show,\nStars and galaxies dance, as the universe grows.'}


In [18]:
print(output['fact'])
print('\n\n')
print(output['poem'])

Here's an overview of the solar system:

• Our solar system consists of eight planets, including Mercury, Mars, Earth, and Jupiter, orbiting around the Sun.

• The four outer planets (Jupiter, Saturn, Uranus, and Neptune) are gas giants with diverse compositions and unique features.



Planets orbit Earth, a celestial show,
Stars and galaxies dance, as the universe grows.


## Chain Router

In [37]:
prompt = """Given the user review below, classify it as either being about `Positive` or `Negative`.'
            Do not respond with moren than one word.

            Review: {review}
            Classification:"""

template = ChatPromptTemplate.from_template(prompt)

chain = template | llm | StrOutputParser()

review = "Thank you so much for providing such a great platform for learning. I am really happy with the service."

# review = "This service is the worst I have ever seen. I will never use it again."

chain.invoke({'review': review})

'Negative.'

In [41]:
positive_review = """
You are an expert in writing reply for positive reviews.
You need to encourage the user to share their experience in social media.
Review: {review}
Answer:"""

positive_prompt = ChatPromptTemplate.from_template(positive_review)

positive_chain = positive_prompt | llm | StrOutputParser()


In [39]:
negative_review = """
You are an expert in writing reply for negative reviews.
You need first to apologize for the inconvenience caused to the users.
You need to encourage the user to share their concerns on following email: 'letstalk@abc.xyz'
Review: {review}
Answer:"""

negative_prompt = ChatPromptTemplate.from_template(negative_review)

negative_chain = negative_prompt | llm | StrOutputParser()

In [40]:
def route(info):
  if 'positive' in info['sentiment'].lower():
    return positive_chain
  else:
    return negative_chain

In [33]:
# route({'sentiment': 'negative'})

In [42]:
from langchain_core.runnables import RunnableLambda

In [44]:
full_chain = {'sentiment': chain, 'review': lambda x: x['review']} | RunnableLambda(route) | StrOutputParser()

In [48]:
# review = "Thank you so much for providing such a great platform for learning. I am really happy with the service."

review = "This service is the worst I have ever seen. I will never use it again."

output = full_chain.invoke({'review': review})

print(output)

Here's a potential response:

Dear valued customer,

I want to start by apologizing for the disappointing experience you had with our service. We regret that we failed to meet your expectations, and for that, we are truly sorry.

We understand that choosing a service can be a frustrating and costly decision, and it's unacceptable that ours fell short of your needs. Please know that this is not a reflection on the hard work and dedication of our team.

In an effort to prevent similar issues in the future, we would like to thank you for taking the time to share your feedback with us. Your input will be invaluable in helping us improve our services and ensure that others have a better experience.

If you would be willing, could you please reach out to us at letstalk@abc.xyz so we can discuss how we can make things right? We are committed to providing excellent service and value to all our customers.

Thank you for your time, and we hope to have the opportunity to serve you better in the f

## Make Custom Chain Runnables with RunnablePassthrough and RunnableLambda

In [1]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

In [19]:
def char_counts(text):
  return len(text)

def word_counts(text):
  return len(text.split())

prompt = ChatPromptTemplate.from_template('Explain these inputs in 5 sentences: {input1} and {input2}')

In [18]:
chain = prompt | llm | StrOutputParser() 

output = chain.invoke({'input1': 'The sun is a star.', 'input2': 'The moon is a satellite.'})

print(output)

These are two distinct concepts that need clarification.

The sun is indeed a star, but it's the center of our solar system. It's a massive ball of hot, glowing gas that generates its own light and heat through nuclear reactions in its core. The sun is a massive celestial body that supports life on Earth and is the primary source of energy for our planet.

On the other hand, the moon is not technically a satellite, but rather a natural satellite that orbits the Earth. It's a rocky, airless body that was once part of the Earth's mantle but was broken off and later coaxed into orbit by the gravitational force of the Earth. The moon is about 2,159 miles (3,475 kilometers) away from our planet and has its own unique characteristics, such as tides and phases.

Here are a few key differences between a star like the sun and objects in space that are often referred to as "sats" or satellites:

* A star is a massive ball of hot gas with intense energy.
* Objects in space that are not stars (lik

In [21]:
chain = prompt | llm | StrOutputParser() | {'char_counts': RunnableLambda(char_counts), 'word_counts': RunnableLambda(word_counts), 'output': RunnablePassthrough()}

output = chain.invoke({'input1': 'The sun is a star.', 'input2': 'The moon is a satellite.'})

print(output)

{'char_counts': 714, 'word_counts': 119, 'output': 'The word "sun" refers to the celestial body that provides light and warmth to our planet, primarily through nuclear reactions that occur in its core. On the other hand, the term "moon" describes a natural satellite that orbits a celestial body, often gravitationally bound to it, and is typically much smaller than Earth. In this context, the moon is not a star itself but rather a distinct object that exists independently of the sun. The concept of a moon and a sun has been observed in our solar system for centuries, with some moons orbiting very close to their parent planets. The distinction between these two terms highlights the unique characteristics and relationships of celestial bodies in our universe.'}
